In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import collections

import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import string
import math

In [26]:
df = pd.read_csv("df_date.csv", sep="\t", encoding="utf-8")
df_test_topic = df[276804:]

,Unnamed: 0.1,Unnamed: 0,abstract,author,content,date,source,tags,title,topic,url
0,101366,101366,KKTC'de dağılan koalisyonun ortağı lideri Denk...,NaN,KKTC'de yaşanan siyasi çalkantı sonucu dağılan...,2006-01-10 11:34:00,haberturk,[],AKP 'Babanı sustur' dedi,Gündem,https://www.haberturk.com/gundem/haber/1717-ak...
1,67804,67804,"İngiliz uzmanlar, grip ilaçlarının, tedavisi i...",NaN,"İngiltere'deki sağlık otoriteleri, grip tedavi...",2006-01-10 19:02:00,haberturk,[],Dikkat! 'Süper Virüsler' çok tehlikeli,Sağlık,https://www.haberturk.com/saglik/haber/1746-di...
2,46689,46689,"'Eve Dönüş' filmi sadece işkence değil, ateşli...",NaN,Memet Ali Alabora ile Sibel Kekilli'nin yarı ç...,2006-01-11 00:10:00,haberturk,[],Bu filmde çıplaklık yarım erotizm tam,Kültür-Sanat,https://www.haberturk.com/kultur-sanat/haber/4...
3,265931,265931,"BU sene modacı Zac Posen ile çalışan Vakko, bu...",NaN,"Vakko, bu sezon ünlü New York’lu moda tasarımc...",2006-02-10 22:55:00,haberturk,[],Dita soyundu kadınlar çığlık attı,Magazin,https://www.haberturk.com/haber/haber/1795-dit...
4,226189,226189,Bombalama ve terör olayları uçuşlarda özel jet...,NaN,LONDRA HAVAALANLARINDA YAŞANAN ŞAŞIRTMACALI bo...,2006-03-10 12:18:00,haberturk,[],Özel jet satışları patladı,Dünya,https://www.haberturk.com/dunya/haber/1906-oze...
...,...,...,...,...,...,...,...,...,...,...,...
307557,135509,135509,"Putin, ""Güvenlik ve askeri işbirliği konuların...",NaN,"Rusya Devlet Başkanı Vladimir Putin, güvenlik ...",2019-12-09 22:26:00,"AP, Anadolu Ajansı","['Dünya', 'Rusya', 'İsrail', 'Vladimir Putin']",“Rusya-İsrail ilişkileri yeni nitelik kazandı”,Dünya,https://www.ntv.com.tr/dunya/rusya-israil-ilis...
307558,1534,1534,Uyuşturucu kaçakçısı mahkumun firar etmesi üze...,NaN,"Güney Amerika ülkesi Paraguay'da, uyuşturucu k...",2019-12-09 22:49:00,"AP, Anadolu Ajansı","['Dünya', 'uyuşturucu', 'Güney Amerika']",Paraguay'ı karıştıran firar: Bakan istifa etti...,Dünya,https://www.ntv.com.tr/dunya/paraguayi-karisti...
307559,64651,64651,Kocaeli'nin Körfez ilçesinde park halindeki ik...,AA,'nin Körfez ilçesinde; önceki akşam Mimar Sina...,2019-12-09 23:07:00,haberturk,"['kocaeli', 'kocaeli silahlı saldırı']",Kocaeli'de iki otomobil kurşunlandı,Gündem,https://www.haberturk.com/kocaeli-de-iki-otomo...
307560,193855,193855,Avrupa Birliği (AB) Komisyonu Başkanı Jean-Cla...,NaN,Avrupa Birliği (AB) Komisyonu Başkanı Jean-Cla...,2019-12-09 23:28:00,"AP, Anadolu Ajansı","['Dünya', 'İngiltere', 'Avrupa Birliği']","""İngilizler başından beri yarı zamanlı Avrupal...",Dünya,https://www.ntv.com.tr/dunya/ingilizler-basind...


In [29]:
print(*df_test_topic['topic'].unique(), sep='\n')

Türkiye
turkiye
Spor
Dünya
Otomobil
Sağlık
Gündem
Teknoloji
Ekonomi
Yaşam
Sanat
futbol
BBC
Televizyon
nan
basketbol
Seyahat
kultur-sanat
ekonomi
siyaset
Eğitim
cumhuriyet_pazar
Sosyal Güvenlik
Futbol
yerel_yonetimler
dunya
2019 Yerel Seçim
Kültür-Sanat
spor
bilim_ve_teknoloji
İş-Yaşam
Magazin
diger_sporlar
yasam
Altın
cumhuriyet_ege
Emlak
Enerji
Sigorta
yazi_dizileri
Para
Fiskos
Müzik
Tenis
Kadına Şiddet
Özel Röportajlar
Basketbol
Nedir
Döviz
etkinlik
Merak Edilenler
Yemek Yapma
Nasıl Yapılır
kitap
Turizm
pazar_yazilari
egitim
cevre
saglik
Tarifler
Alışveriş
Savunma Sanayi
uzay
Borsa
Bitcoin
Diğer
calisma_yasami
23 Haziran seçimleri
Ramazan
Girişimcilik
Güvenli Alışveriş
Dünyadan
MOTY 2019
Akdeniz Ekonomi Forumu
konser
Voleybol
Motor Sporları
otomobil
Biyografi
Evde Ek İş
sinema
Şipşak
gezi
secim
diger


In [30]:
def frequency_dict(dataframe):
    
    topic_list = list(dataframe['topic'])
    dict_freq = dict(collections.Counter(topic_list))
    print(*dict_freq.items(), sep='\n')

In [31]:
df_renamed = df_test_topic

df_renamed.loc[df_renamed.topic == "Futbol", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "futbol", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Basketbol", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Tenis", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Hentbol", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Olimpiyat", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "spor", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Motor Sporları", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Voleybol", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "2016 Avrupa Şampiyonası", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "foto_spor", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "EURO 2016", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "2016 Rio Olimpiyatları", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "2018 Dünya Kupası", "topic"] = "Spor"
df_renamed.loc[df_renamed.topic == "Dünya Kupası 2018", "topic"] = "Spor"

df_renamed.loc[df_renamed.topic == "turkiye", "topic"] = "Türkiye"

df_renamed.loc[df_renamed.topic == "Fiskos", "topic"] = "Magazin"

df_renamed.loc[df_renamed.topic == "ekonomi", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Makro Ekonomi", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Para", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Döviz", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Borsa", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Altın", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Ege Ekonomik Forum 2017", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Bitcoin", "topic"] = "Ekonomi"
df_renamed.loc[df_renamed.topic == "Akdeniz Ekonomi Forumu", "topic"] = "Ekonomi"

df_renamed.loc[df_renamed.topic == "dunya", "topic"] = "Dünya"
df_renamed.loc[df_renamed.topic == "Dünyadan", "topic"] = "Dünya"

df_renamed.loc[df_renamed.topic == "yasam", "topic"] = "Yaşam"

df_renamed.loc[df_renamed.topic == "Sanat", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "Kültür-Sanat", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "kultur-sanat", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "sinema", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "kitap", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "Müzik", "topic"] = "Kültür_Sanat"
df_renamed.loc[df_renamed.topic == "konser", "topic"] = "Kültür_Sanat"

df_renamed.loc[df_renamed.topic == "saglik", "topic"] = "Saglık"

df_renamed.loc[df_renamed.topic == "bilim_ve_teknoloji", "topic"] = "Teknoloji"

df_renamed.loc[df_renamed.topic == "egitim", "topic"] = "Eğitim"

df_renamed.loc[df_renamed.topic == "siyaset", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "yerel_yonetimler", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "2019 Yerel Seçim", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "Seçim 2015", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "secim_2015", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "Seçim", "topic"] = "Siyaset"
df_renamed.loc[df_renamed.topic == "secim", "topic"] = "Siyaset"

df_renamed.loc[df_renamed.topic == "Kadına Şiddet", "topic"] = "Kadına_Şiddet"

df_renamed.loc[df_renamed.topic == "otomobil", "topic"] = "Otomobil"

df_renamed.loc[df_renamed.topic == "gezi", "topic"] = "Seyahat"

df_renamed.loc[df_renamed.topic == "calisma_yasami", "topic"] = "İş"
df_renamed.loc[df_renamed.topic == "İş-Yaşam", "topic"] = "İş"

df_renamed.loc[df_renamed.topic == "cevre", "topic"] = "Çevre"
df_renamed.loc[df_renamed.topic == "Sıfır Atık", "topic"] = "Çevre"

df_renamed.loc[df_renamed.topic == "Yemek Yapma", "topic"] = "Mutfak"
df_renamed.loc[df_renamed.topic == "Tarifler", "topic"] = "Mutfak"

df_renamed.loc[df_renamed.topic == "Özel Röportajlar", "topic"] = "Röportaj"
df_renamed.loc[df_renamed.topic == "Röportajlar", "topic"] = "Röportaj"

df_renamed = df_renamed.loc[df_renamed['topic'].isin(['Spor', 'Türkiye', 'Magazin', 'Ekonomi', 
                                         'Dünya', 'Medya', 'Yaşam', 'Kültür_Sanat', 
                                         'Sağlık', 'Teknoloji', 'Eğitim', 'Siyaset', 
                                         'Enerji', 'Yemek', 'Kadına_Şiddet', 'Otomobil', 
                                         'Televizyon', 'Seyahat', 'Polemik', 'Turizm', 
                                         'İş', 'Emlak', 'Çevre'])]

In [38]:
df_renamed = df_renamed.loc[df_renamed['topic'].isin(['Magazin', 'Ekonomi', 'Dünya', 
                                                   'Kültür_Sanat', 'Sağlık', 'Spor', 
                                                   'Teknoloji', 'Eğitim', 'Siyaset', 'Otomobil'])]

frequency_dict(df_renamed)

('Spor', 2594)
('Dünya', 5387)
('Otomobil', 87)
('Sağlık', 920)
('Teknoloji', 991)
('Ekonomi', 1725)
('Kültür_Sanat', 1275)
('Siyaset', 780)
('Eğitim', 434)
('Magazin', 281)


In [40]:
len(df_renamed)

14474

In [41]:
df_renamed = df_renamed[['abstract', 'title', 'topic']]

,abstract,title,topic
276806,Sakarya’nın Serdivan ilçesinde Serdivanspor il...,Sahada başlayan kavga tribüne sıçradı (8 kırmı...,Spor
276807,"Avustralya Başbakanı Scott Morrison, merkezde ...",Nauru Gözaltı Merkezi'nde sığınmacı çocuk kalm...,Dünya
276808,"SEAT, soğuk kış günlerinde otomobil camlarında...",SEAT’tan kış ayları için 4 öneri,Otomobil
276811,"İlerleyen yaşla birlikte, cildin elastikiyetin...",Kış güneşi cildinizi hızla yaşlandırıyor!,Sağlık
276813,"Sudan Dışişleri Bakanı Ahmed, OAC hükümeti ile...",OAC'de nihai barış anlaşması çarşamba imzalanacak,Dünya
...,...,...,...
307555,"Ülke parlamentosunda milletvekilleri, binlerce...",Arjantin'de sokak gösterilerine neden olan 'Ac...,Dünya
307557,"Putin, ""Güvenlik ve askeri işbirliği konuların...",“Rusya-İsrail ilişkileri yeni nitelik kazandı”,Dünya
307558,Uyuşturucu kaçakçısı mahkumun firar etmesi üze...,Paraguay'ı karıştıran firar: Bakan istifa etti...,Dünya
307560,Avrupa Birliği (AB) Komisyonu Başkanı Jean-Cla...,"""İngilizler başından beri yarı zamanlı Avrupal...",Dünya


In [43]:
# SOURCE: https://github.com/apdullahyayik/TrTokenizer

from trtokenizer.tr_tokenizer import SentenceTokenizer, WordTokenizer
word_tokenizer_object = WordTokenizer()

In [44]:
def clean_text(text):
    
    text = text.lower()   
    text = text.translate(str.maketrans('', '', string.punctuation)) # !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~
    words = word_tokenizer_object.tokenize(text)
    stopWords = set(stopwords.words('turkish'))
    new_text = ' '.join(word for word in words if word not in stopWords)

    return new_text

In [47]:
# clean all texts
df_renamed.abstract = [clean_text(text) for text in df_renamed.abstract]
df_renamed.to_csv("test_10_topic.csv", sep='\t')

,abstract,title,topic
276806,sakarya’nın serdivan ilçesinde serdivanspor ka...,Sahada başlayan kavga tribüne sıçradı (8 kırmı...,Spor
276807,avustralya başbakanı scott morrison merkezde k...,Nauru Gözaltı Merkezi'nde sığınmacı çocuk kalm...,Dünya
276808,seat soğuk kış günlerinde otomobil camlarındak...,SEAT’tan kış ayları için 4 öneri,Otomobil
276811,i lerleyen yaşla birlikte cildin elastikiyetin...,Kış güneşi cildinizi hızla yaşlandırıyor!,Sağlık
276813,sudan dışişleri bakanı ahmed oac hükümeti sila...,OAC'de nihai barış anlaşması çarşamba imzalanacak,Dünya
...,...,...,...
307555,ülke parlamentosunda milletvekilleri binlerce ...,Arjantin'de sokak gösterilerine neden olan 'Ac...,Dünya
307557,putin güvenlik askeri işbirliği konularında i ...,“Rusya-İsrail ilişkileri yeni nitelik kazandı”,Dünya
307558,uyuşturucu kaçakçısı mahkumun firar etmesi üze...,Paraguay'ı karıştıran firar: Bakan istifa etti...,Dünya
307560,avrupa birliği ab komisyonu başkanı jeanclaude...,"""İngilizler başından beri yarı zamanlı Avrupal...",Dünya
